In [153]:
import pandas as pd

df_X = pd.read_csv('X_NorthAmerica.csv')
df_X.head()


,CompNo,yyyy,mm,StkIndx,STInt,dtdlevel,dtdtrend,liqnonfinlevel,liqnonfintrend,ni2talevel,ni2tatrend,sizelevel,sizetrend,m2b,sigma,liqfinlevel,lqfintrend,DTDmedianFin,DTDmedianNonFin,dummy297fin
0,15984000.0,1999.0,3,0.167570,0.006248,NaN,NaN,NaN,NaN,NaN,NaN,1.027931,0.000000,NaN,NaN,0.0,0.0,0.0,2.324205,0
1,15984000.0,1999.0,4,0.200972,0.006487,NaN,NaN,NaN,NaN,NaN,NaN,1.351854,0.323923,NaN,NaN,0.0,0.0,0.0,2.373554,0
2,15984000.0,1999.0,5,0.186079,0.006896,NaN,NaN,NaN,NaN,NaN,NaN,1.358506,0.013304,NaN,0.274280,0.0,0.0,0.0,2.401488,0
3,15984000.0,1999.0,6,0.210673,0.007457,NaN,NaN,0.447325,0.0,0.00932,0.0,1.362518,0.012036,5.082778,0.276896,0.0,0.0,0.0,2.455890,0
4,15984000.0,1999.0,7,0.162536,0.007322,4.190356,0.0,0.447325,0.0,0.00932,0.0,1.336588,-0.103719,4.504762,0.259755,0.0,0.0,0.0,2.455596,0


In [154]:
## preprocess the X dataset

df_X['yyyy'] = df_X['yyyy'].astype(int)
df_X['mm'] = df_X['mm'].astype(str)

for index, row in df_X.iterrows():
    if len(row['mm']) == 1:  
        df_X.at[index, 'EventDate'] = str(row['yyyy']) + '0' + row['mm']
    else:
        df_X.at[index, 'EventDate'] = str(row['yyyy']) + row['mm']

    df_X.at[index, 'CompNo'] = row['CompNo']/1000
df_X['CompNo'] = df_X['CompNo'].astype(int)

# Convert the "EventDate" column to datetime format
df_X['EventDate'] = pd.to_datetime(df_X['EventDate'], format='%Y%m')

# Format the "EventDate" column to show year and month (yyyy mm)
df_X['EventDate'] = df_X['EventDate'].dt.strftime('%Y %m')df_X.head()

# Rename the "CompNo" column to "CompanyNumber"
df_X = df_X.rename(columns={'CompNo': 'CompanyNumber'})
df_X.head()

C:\Users\shiho\AppData\Local\Temp\ipykernel_42264\1864653634.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '199903' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_X.at[index, 'EventDate'] = str(row['yyyy']) + '0' + row['mm']


,CompNo,yyyy,mm,StkIndx,STInt,dtdlevel,dtdtrend,liqnonfinlevel,liqnonfintrend,ni2talevel,...,sizelevel,sizetrend,m2b,sigma,liqfinlevel,lqfintrend,DTDmedianFin,DTDmedianNonFin,dummy297fin,EventDate
0,15984,1999,3,0.167570,0.006248,NaN,NaN,NaN,NaN,NaN,...,1.027931,0.000000,NaN,NaN,0.0,0.0,0.0,2.324205,0,199903
1,15984,1999,4,0.200972,0.006487,NaN,NaN,NaN,NaN,NaN,...,1.351854,0.323923,NaN,NaN,0.0,0.0,0.0,2.373554,0,199904
2,15984,1999,5,0.186079,0.006896,NaN,NaN,NaN,NaN,NaN,...,1.358506,0.013304,NaN,0.274280,0.0,0.0,0.0,2.401488,0,199905
3,15984,1999,6,0.210673,0.007457,NaN,NaN,0.447325,0.0,0.00932,...,1.362518,0.012036,5.082778,0.276896,0.0,0.0,0.0,2.455890,0,199906
4,15984,1999,7,0.162536,0.007322,4.190356,0.0,0.447325,0.0,0.00932,...,1.336588,-0.103719,4.504762,0.259755,0.0,0.0,0.0,2.455596,0,199907


In [6]:
df_X.shape[0]

2753077

In [97]:
import pandas as pd
from datetime import datetime, timedelta

df_Y = pd.read_csv('Y_NorthAmerica.csv')
df_Y = df_Y.rename(columns={'CompNo': 'CompanyNumber', 'Event_type': 'EventType'})
df_Y = df_Y.drop(['Sector_Number', 'Exchange'], axis=1)

df_Y_new = pd.DataFrame()

# Convert the "EventDate" column to datetime format
df_Y['EventDate'] = pd.to_datetime(df_Y['EventDate'], format='%Y%m%d')

# Format the "EventDate" column to show year and month (yyyy mm)
df_Y['EventDate'] = df_Y['EventDate'].dt.strftime('%Y %m')

# Sort the DataFrame by CompanyNumber and EventDate
df_Y.sort_values(by=['CompanyNumber', 'EventDate'], inplace=True)

# Function to check if event type 1 occurred n months ago for the same comp number
def check_n_months_ago(df, n):
    global df_Y_new  # Declare df_Y_new as a global variable

    for index, row in df.iterrows():  
        comp_number = row['CompanyNumber']
        event_date = row['EventDate']

        # Reset the index 
        df = df.reset_index(drop=True)

        if row['EventType'] == 1:
            for i in range(1, n+1):  # Check for the past n months
                # Convert event_date to datetime format
                event_date = pd.to_datetime(event_date, format='%Y %m')
                # Calculate the date i months ago from the current event date
                i_months_ago = event_date - pd.DateOffset(months=i)

                # Check if the combination of CompanyNumber and i_months_ago is not in the original df_Y
                if not ((df['CompanyNumber'] == comp_number) & (df['EventDate'] == i_months_ago.strftime('%Y %m'))).any():
                    # If not present, add a new row to df_Y_new
                    new_row = {'EventDate': i_months_ago.strftime('%Y %m'), 'CompanyNumber': comp_number, 'EventType': 1}
                    new_df = pd.DataFrame([new_row])
                    df_Y_new = pd.concat([df_Y_new, new_df], ignore_index=True)

    return df_Y_new

# Call the function for the past 12 months (n = 12)
result_df = check_n_months_ago(df_Y, 12)

new_df = pd.concat([df_Y, result_df], ignore_index=True)
new_df = new_df.drop_duplicates(subset=['EventDate', 'CompanyNumber','EventType'], keep='last')

for index, row in new_df.iterrows():
    new_df.at[index, 'CompanyNumber'] = row['CompanyNumber']/1000
new_df['CompanyNumber'] = new_df['CompanyNumber'].astype(int)

new_df

       CompanyNumber EventDate  EventType
0            7998000   2023 07          0
1            7998001   2019 04          2
2            7998002   2011 07          2
3           14547000   2016 11          2
4           14547001   2009 06          2
...              ...       ...        ...
49859      209445000   2022 06          1
49860      209445000   2022 05          1
49861      209445000   2022 04          1
49862      209445000   2022 03          1
49863      209445000   2022 02          1

[49864 rows x 3 columns]


In [165]:
new_df.shape[0]

49864

In [162]:
from pandas_gbq import to_gbq
import json

result = pd.merge(df_X, new_df, on=['CompanyNumber', 'EventDate'], how='left')

result['EventType'] 

result.to_csv('X_Y_merged.csv', index=False)
result['EventType'] = result['EventType'].fillna(0)
result

,CompanyNumber,yyyy,mm,StkIndx,STInt,dtdlevel,dtdtrend,liqnonfinlevel,liqnonfintrend,ni2talevel,...,sizetrend,m2b,sigma,liqfinlevel,lqfintrend,DTDmedianFin,DTDmedianNonFin,dummy297fin,EventDate,EventType
0,15984,1999,3,0.167570,0.006248,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,0.0,0.0,0.000000,2.324205,0,1999 03,0.0
1,15984,1999,4,0.200972,0.006487,NaN,NaN,NaN,NaN,NaN,...,0.323923,NaN,NaN,0.0,0.0,0.000000,2.373554,0,1999 04,0.0
2,15984,1999,5,0.186079,0.006896,NaN,NaN,NaN,NaN,NaN,...,0.013304,NaN,0.274280,0.0,0.0,0.000000,2.401488,0,1999 05,0.0
3,15984,1999,6,0.210673,0.007457,NaN,NaN,0.447325,0.0,0.00932,...,0.012036,5.082778,0.276896,0.0,0.0,0.000000,2.455890,0,1999 06,0.0
4,15984,1999,7,0.162536,0.007322,4.190356,0.0,0.447325,0.0,0.00932,...,-0.103719,4.504762,0.259755,0.0,0.0,0.000000,2.455596,0,1999 07,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2753072,212037,2023,7,0.015960,-0.010290,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,0.0,0.0,0.000000,2.087496,0,2023 07,0.0
2753073,212083,2023,6,0.038825,-0.010290,NaN,NaN,0.000000,0.0,NaN,...,0.000000,NaN,NaN,NaN,NaN,3.405230,0.000000,1,2023 06,0.0
2753074,212083,2023,7,0.015960,-0.010290,NaN,NaN,0.000000,0.0,NaN,...,-0.723197,NaN,NaN,NaN,NaN,3.453191,0.000000,1,2023 07,0.0
2753075,212122,2023,7,0.015960,-0.010290,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,0.0,0.0,0.000000,2.087496,0,2023 07,0.0


In [163]:
with open('../token.json', 'r') as token_file:
    token_data = json.load(token_file)
    project_id = token_data.get('project_id', 'default-project-id')

dataset_id = "capstone"
table_id = 'X_Y_merged_2'
x_id = 'X_NorthAmerica'
y_id = 'Y_NorthAmerica'


to_gbq(result, destination_table=f'{project_id}.{dataset_id}.{table_id}', project_id=project_id, if_exists='replace')

In [164]:
result.isna().sum().sort_values(ascending=False)

dtdlevel           415711
dtdtrend           415711
liqnonfinlevel     313471
liqnonfintrend     313471
m2b                304269
ni2talevel         295766
ni2tatrend         295766
sigma              289737
sizetrend          160735
sizelevel          160735
DTDmedianNonFin    126687
liqfinlevel         64819
lqfintrend          64819
StkIndx             47043
DTDmedianFin        29918
STInt               17318
dummy297fin             0
EventDate               0
CompanyNumber           0
yyyy                    0
mm                      0
EventType               0
dtype: int64

In [143]:
count_event_type_1 = len(result[result['EventType'] == 1.0])
count_event_type_1

16252

In [17]:
result.shape
result

,CompNo,yyyy,mm,StkIndx,STInt,dtdlevel,dtdtrend,liqnonfinlevel,liqnonfintrend,ni2talevel,...,sigma,liqfinlevel,lqfintrend,DTDmedianFin,DTDmedianNonFin,dummy297fin,EventDate,Event_type,Sector_Number,Exchange
0,15984,1999,3,0.167570,0.006248,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.000000,2.324205,0,199903,NaN,NaN,NaN
1,15984,1999,4,0.200972,0.006487,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.000000,2.373554,0,199904,NaN,NaN,NaN
2,15984,1999,5,0.186079,0.006896,NaN,NaN,NaN,NaN,NaN,...,0.274280,0.0,0.0,0.000000,2.401488,0,199905,NaN,NaN,NaN
3,15984,1999,6,0.210673,0.007457,NaN,NaN,0.447325,0.0,0.00932,...,0.276896,0.0,0.0,0.000000,2.455890,0,199906,NaN,NaN,NaN
4,15984,1999,7,0.162536,0.007322,4.190356,0.0,0.447325,0.0,0.00932,...,0.259755,0.0,0.0,0.000000,2.455596,0,199907,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2753075,212037,2023,7,0.015960,-0.010290,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.000000,2.087496,0,202307,0.0,18.0,16.0
2753076,212083,2023,6,0.038825,-0.010290,NaN,NaN,0.000000,0.0,NaN,...,NaN,NaN,NaN,3.405230,0.000000,1,202306,NaN,NaN,NaN
2753077,212083,2023,7,0.015960,-0.010290,NaN,NaN,0.000000,0.0,NaN,...,NaN,NaN,NaN,3.453191,0.000000,1,202307,0.0,14.0,16.0
2753078,212122,2023,7,0.015960,-0.010290,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.000000,2.087496,0,202307,0.0,12.0,16.0
